In [0]:
%tensorflow_version 1.x

In [0]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.metrics
from sklearn import tree
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2

import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import torch.nn as nn
import os
from PIL import Image
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
file_name = 'categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = np.array(classes)

In [6]:
arch = 'resnet50'
model_file = f'{arch}_places365.pth.tar'
if not os.access(model_file, os.W_OK):
    weight_url = 'http://places2.csail.mit.edu/models_places365/' + model_file
    os.system('wget ' + weight_url)

model = models.__dict__[arch](num_classes=365)
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)

<All keys matched successfully>

In [7]:
model.eval()
for param in model.parameters():
    param.requires_grad = False
'convert model to evaluation mode with no grad'

'convert model to evaluation mode with no grad'

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
feature_extractor = nn.Sequential(*list(model.children())[:-2], nn.AdaptiveAvgPool2d(1)).eval().to(device)

In [0]:
!cp drive/'My Drive'/'Colab Notebooks'/'Automate local TCAV'/'final notebooks'/resized_imgs.pkl  /content/
!cp drive/'My Drive'/'Colab Notebooks'/'Automate local TCAV'/'final notebooks'/masks.pkl  /content/
!cp drive/'My Drive'/'Colab Notebooks'/'Automate local TCAV'/'final notebooks'/classes.pkl  /content/

In [0]:
import pickle
with open('masks.pkl', 'rb') as f:
  masks = pickle.load(f)
with open('resized_imgs.pkl', 'rb') as f:
  imgs = pickle.load(f)
with open('classes.pkl', 'rb') as f:
  labels = np.array(pickle.load(f))

In [0]:
def get_segments(img, mask, threshold = 0.05):
  ade_classes = pd.read_csv('https://raw.githubusercontent.com/CSAILVision/sceneparsing/master/objectInfo150.csv')
  segs = np.unique(mask)
  segments = []
  total = mask.shape[0]*mask.shape[1]
  segments_classes = []
  for seg in segs:
    idxs = mask==seg
    sz = np.sum(idxs)
    if sz < threshold*total:
      continue
    segment = img*idxs[..., None]
    w, h, _ = np.nonzero(segment)
    segment = segment[np.min(w):np.max(w),np.min(h):np.max(h),:]
    segments.append(segment)
    segments_classes.append(ade_classes['Name'].loc[ade_classes['Idx']==seg].iloc[0])
  return segments, segments_classes

In [0]:
img_segments = []
img_segments_classes = []
for img, msk in zip(imgs, masks):
  segss, seg_class = get_segments(np.array(img), msk, threshold = 0.005)
  img_segments_classes.append(seg_class)
  img_segments.append(segss)

In [0]:
centre_crop = trn.Compose([
        trn.Resize((256,256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        # trn.Normalize([0, 0, 0], [255, 255, 225]),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [15]:
outputs = []
def hook(module, input, output):
    x = nn.AdaptiveAvgPool2d(1)(output)
    outputs.append(x.cpu().numpy().squeeze())
full_model = model
handle = full_model.layer4[2].register_forward_hook(hook)
full_model.to(device)
'finished the full model with hook attached'

'finished the full model with hook attached'

In [0]:
chosen_classes = ['street', 'bedroom', 'living_room', 'bathroom', 'kitchen', 
            'skyscraper', 'highway', 'conference_room', 'mountain_snowy', 'office',
            'corridor', 'airport_terminal', 'attic', 'mountain', 'park', 'coast', 
            'alley','beach', 'childs_room', 'art_gallery','castle', 'dorm_room', 
            'nursery', 'lobby', 'reception', 'bar', 'house', 'bridge', 'classroom']
num_classes = len(chosen_classes)
idxs = []
c = 0
for ccls in chosen_classes:
  idx = np.argwhere(classes == ccls)
  if len(idx) == 0:
    print(f'class {ccls} is not found in places365, so we will use places365 alternate')
    c+=1
  else:
    idxs.append(idx[0][0])

In [0]:
del outputs
outputs = []
y_model = []
for img in imgs:
  input_img = V(centre_crop(img).unsqueeze(0))
  input_img = input_img.to(device)
  pred = full_model.forward(input_img)
  y_model.append(classes[idxs][np.argmax((pred.cpu().detach().numpy()[:,idxs]))])
outputs = np.array(outputs)
img_vectors = np.copy(outputs)

In [0]:
del outputs
outputs = []
img_segmentsid = []
segments_classes = []
for i, img_seg in enumerate(img_segments):
    img_segmentsid.append((img_segmentsid[-1] if i>0 else 0) + len(img_seg))
    for seg in img_seg:
      img = Image.fromarray(seg, 'RGB')
      input_img = V(centre_crop(img).unsqueeze(0))
      input_img = input_img.to(device)
      pred = full_model.forward(input_img)
      segments_classes.append(classes[idxs][np.argmax((pred.cpu().detach().numpy()[:,idxs]))])
outputs = np.array(outputs)
all_vectors = np.copy(outputs)

In [0]:
feature_vectors = []
for i in range(len(img_segmentsid)):
  feature_vectors.append(all_vectors[(img_segmentsid[i-1] if i>0 else 0):img_segmentsid[i]])

In [0]:
segment_img = {}
c = 0
for j, fvec in enumerate(feature_vectors): 
  c_old = c
  c += len(fvec) if len(fvec) != 1024 else 1
  for i in range(c_old, c):
    segment_img[i] = j

In [0]:
import sklearn.metrics
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [0]:
def cluster_top_k(cluster_id, k = 40):
  instances_ids = np.argwhere(clusters_labels==cluster_id).squeeze()
  instances = X[instances_ids]
  scores = []
  for inst in instances:
    scores.append(-kmeans.score([inst]))
  final_ids = np.argsort(scores)[:k]
  return instances_ids[final_ids]

cluster_instances_id = lambda cid:  np.argwhere(clusters_labels==cid).squeeze()

def is_single_img_clusters(cls_id):
  instances_ids = cluster_instances_id(cls_id)
  # instances_ids = cluster_representatives[cls_id]
  source_imgs = [segment_img[inst_id] for inst_id in instances_ids]
  unique_imgs = len(np.unique(source_imgs))
  return 0 if unique_imgs > 1 else 1

In [0]:
img_distances = sklearn.metrics.pairwise.euclidean_distances(img_vectors)

In [24]:
num_neighbors = 300
train_accuracy = {}
test_accuracy = {}
train_f1 = {}
test_f1 = {}
hit = {}
for i in [0,2,5,8,10]:
  train_accuracy[i] = []
  test_accuracy[i] = []
  train_f1[i] = []
  test_f1[i] = []
  hit[i] = []

# for instance_id in range(len(img_vectors)):
for instance_id in range(500):
  neighbors = np.argsort(img_distances[instance_id])[:num_neighbors]
  neighbor_vectors = img_vectors[neighbors]
  neighbor_labels = np.array(y_model)[neighbors]
  print('clustering the neighbor dataset ................')
  X = np.array([v for i in neighbors for v in feature_vectors[i]])
  num_segs = X.shape[0]
  k = int(num_segs**0.5)
  kmeans = KMeans(n_clusters=k, random_state=0).fit(X)
  clusters_labels = kmeans.labels_
  print('\ndone')

  cluster_names, cluster_counts = np.unique(clusters_labels, return_counts=1)
  print('removing small clusters  ................')
  count_threshold = int(k*0.6)
  ids = cluster_counts > count_threshold
  good_clusters = cluster_names[ids]
  cluster_counts = cluster_counts[ids]
  print(f'\ntotal number of clusters {len(cluster_names)}')
  print(f'total number of good clusters is {len(good_clusters)}')
  print('\ndone')

  print('calculating the clusters importances ................')
  cluster_scores = []
  for cls in good_clusters:
    cluster_scores.append(kmeans.score(X[np.argwhere(clusters_labels==cls).squeeze()]))
  cluster_scores = np.array(cluster_scores)
  cluster_importance = [cluster_counts, cluster_scores]
  cluster_importance.append(cluster_importance[1]/cluster_importance[0])
  print('\ndone')

  print('removing single image clusters ............')
  final_clusters = []
  to_remove = []
  for j, cluster_id in enumerate(good_clusters):
    if is_single_img_clusters(cluster_id):
      to_remove.append(j)
      continue
    final_clusters.append(cluster_id)
  for i in range(len(cluster_importance)):
    cluster_importance[i] = np.delete(cluster_importance[i],to_remove)
  final_clusters = np.array(final_clusters)
  print('\ndone')

  print('building linear models for each concept/cluster ............')
  linear_models = {}
  model_score = {}
  for cluster in final_clusters:
    positive_instances = cluster_instances_id(cluster)
    negative_instances = np.argwhere(clusters_labels!=cluster).squeeze()
    num_samples = min(len(positive_instances), len(negative_instances))
    selected_pos = np.random.choice(positive_instances, num_samples, replace=False)
    selected_neg = np.random.choice(negative_instances, num_samples, replace=False)
    train_x = np.append(X[selected_pos], X[selected_neg], axis=0)
    train_y = np.array([1] * num_samples + [0] * num_samples)
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y)
    n_components = min(256, len(train_x))
    pca_model = PCA(n_components=n_components)
    lr_model = LogisticRegression()
    pca_lr_model = Pipeline(steps = [('pca', pca_model), ('lr', lr_model)])
    pca_lr_model.fit(train_x, train_y)
    linear_models[cluster] = pca_lr_model
    model_score[cluster] = pca_lr_model.score(val_x, val_y)
  print('\ndone')

  print('removing low scoring clusters based on holdout accuracy ...............\n')
  c = 0
  to_remove = []
  for k, v in list(model_score.items()):
    if v < 0.75:
      c+=1
      print(f'removed cluster {k}')
      linear_models.pop(k, 'None')
      model_score.pop(k, 'None')
      to_remove.append(np.argwhere(final_clusters==k)[0][0])
  for i in range(len(cluster_importance)):
    cluster_importance[i] = np.delete(cluster_importance[i],to_remove)
  print(f'\ndone\nremoved {c} clusters\n')

  print('converting feature vectors to binary concept vectors .................')
  for i, lm in enumerate(linear_models.keys()):
    if i == 0:
      concept_vecs = linear_models[lm].predict(neighbor_vectors)[:, None]
    else:
      concept_vecs = np.concatenate((concept_vecs, linear_models[lm].predict(neighbor_vectors)[:, None]), axis = -1)
  print(f'\nconcept vector dimension is\t {concept_vecs.shape[0]}x{concept_vecs.shape[1]}')
  print(f'original dimension is\t\t {neighbor_vectors.shape[0]}x{neighbor_vectors.shape[1]}')
  print('\ndone')

  print('building surrogate tree .............\n')
  random_seed = 0
  new_concept_vecs = concept_vecs[:, np.argsort(cluster_importance[2])]
  train_x, val_x, train_y, val_y = train_test_split(new_concept_vecs, neighbor_labels, test_size=0.2, random_state=random_seed)
  for i in [0,2,5,8,10]:
    surrogate_tree = DecisionTreeClassifier(max_depth = 20)
    surrogate_tree.fit(train_x[:,i:], train_y)
    train_score = surrogate_tree.score(train_x[:,i:], train_y)
    test_score = surrogate_tree.score(val_x[:,i:], val_y)
    train_f1score = sklearn.metrics.f1_score(train_y, surrogate_tree.predict(train_x[:,i:]), average='weighted')
    test_f1score = sklearn.metrics.f1_score(val_y, surrogate_tree.predict(val_x[:,i:]), average='weighted')
    print(f'train accuracy score after removing {i} concepts:\t{train_score}')
    print(f'test accuracy score after removing {i} concepts:\t {test_score}')
    print(f'train f1 score after removing {i} concepts:\t {train_f1score}')
    print(f'test f1 score after removing {i} concepts:\t {test_f1score}')
    train_accuracy[i].append(train_score)
    test_accuracy[i].append(test_score)
    train_f1[i].append(train_f1score)
    test_f1[i].append(test_f1score)
    hit[i].append(surrogate_tree.predict(new_concept_vecs[0, i:][None, :])[0] == neighbor_labels[0])
    # print(f'hit after removing {i} concepts: \t\t\t {hit[i]}')
  print(f'\ndone\nfinished instance {instance_id}')

Streaming output truncated to the last 5000 lines.
removing low scoring clusters based on holdout accuracy ...............


done
removed 0 clusters

converting feature vectors to binary concept vectors .................

concept vector dimension is	 300x28
original dimension is		 300x2048

done
building surrogate tree .............

train accuracy score after removing 0 concepts:	0.8416666666666667
test accuracy score after removing 0 concepts:	 0.7166666666666667
train f1 score after removing 0 concepts:	 0.8194634746946943
test f1 score after removing 0 concepts:	 0.6861111111111111
train accuracy score after removing 2 concepts:	0.8
test accuracy score after removing 2 concepts:	 0.75
train f1 score after removing 2 concepts:	 0.7686532342105405
test f1 score after removing 2 concepts:	 0.7363247863247864
train accuracy score after removing 5 concepts:	0.7958333333333333
test accuracy score after removing 5 concepts:	 0.7333333333333333
train f1 score after removing 5 concepts:	 0.

In [0]:
with open('train_accuracies.pkl', 'wb') as f:
  pickle.dump(train_accuracy, f)
with open('test_accuracies.pkl', 'wb') as f:
  pickle.dump(test_accuracy, f)
with open('train_f1.pkl', 'wb') as f:
  pickle.dump(train_f1, f)
with open('test_f1.pkl', 'wb') as f:
  pickle.dump(test_f1, f)
with open('hit.pkl', 'wb') as f:
  pickle.dump(hit, f)

In [0]:
!cp train_accuracies.pkl drive/'My Drive'/'Colab Notebooks'/'Automate local TCAV'/'final notebooks'/local_train_accuracies_adding_concepts_final.pkl
!cp test_accuracies.pkl drive/'My Drive'/'Colab Notebooks'/'Automate local TCAV'/'final notebooks'/local_test_accuracies_adding_concepts_final.pkl
!cp train_f1.pkl drive/'My Drive'/'Colab Notebooks'/'Automate local TCAV'/'final notebooks'/local_train_f1_adding_concepts_final.pkl
!cp test_f1.pkl drive/'My Drive'/'Colab Notebooks'/'Automate local TCAV'/'final notebooks'/local_test_f1_adding_concepts_final.pkl
!cp hit.pkl drive/'My Drive'/'Colab Notebooks'/'Automate local TCAV'/'final notebooks'/local_hit_adding_concepts_final.pkl